In [1]:
import bokeh.models as bmodels
import bokeh.plotting as plt
import bokeh.layouts as blay
import bokeh.models.widgets as bmw
import viewer_app.data_source as ds
import pandas as pd

In [5]:


"""Uses data to generate one team graphs.

"""

class OneTeam:

    def __init__(self, data):
        self.measures = data.measures
        self.schedule = data.schedule

    def callback_1t(self, attr, old, new):
        print('In callback for team:', new)
        self.layout_6t.children[1] = self.total_1t(new)

    def df_new_1t(self, team, tasks):
        self.measures = self.measures[(self.measures.team == team) & (self.measures.task.isin(tasks))]
        self.measures.loc[source.measures.capability.isin(['Side', 'Center']), 'successes'] = 5
        self.measures.loc[(source.measures.capability == 'Side'), 'task'] = 'climb_side'
        self.measures.loc[(source.measures.capability == 'Center'), 'task'] = 'climb_center'
        tasks.append('climb_side')
        tasks.append('climb_center')
        tasks.remove('climbPosition')      
        
        # get matches
        grouped = self.measures.groupby(['match', 'task'])
        print(grouped)
        grouped = grouped.sum()
        # grouped = grouped.drop(columns=grouped.columns[1:5])
        grouped = grouped['successes']
        df_unstacked = grouped.unstack()
#         df_unstacked.columns = df_unstacked.columns.droplevel()
        df_fil = df_unstacked.fillna(0)
        return df_fil

    def total_1t(self, team):
        self.cds = bmodels.ColumnDataSource(self.df_new_1t(team))
        tasks = self.cds.column_names
        tasks = tasks[1:]
        matches = self.cds.data['match']
        plt_title = "Team " + team
        colors = ['purple', 'yellow']
        self.pcplot = plt.figure(x_range=matches, plot_height=250,
                                 title=plt_title, tools="hover",
                                 tooltips="$name: @$name")
        self.pcplot.vbar_stack(tasks, x='match', width=0.4,
                               source=self.cds, color=colors,
                          legend_label=[" " + x for x in tasks])
        return self.pcplot

    def list_teams(self):
        return list(self.schedule.team.unique())

    def layout_1t(self, team):
        plot_6t = self.total_1t(team)
        match_sel = bmw.Select(title='Match', options=self.list_teams())
        match_sel.on_change('value', self.callback_1t)
        self.layout_1t = blay.row(match_sel, plot_1t)
        return self.layout_1t

In [6]:
source = ds.DataSource(event='test_event_2', season='2020')
source.measures.loc[source.measures.capability.isin(['Side', 'Center']), 'successes'] = 5
source.measures.loc[source.measures.capability.isin(['Side','Center']), 'task'] = 'climb_' + source.measures.capability
source.measures.query('capability == ["Side", "Center"]')

,date,event,season,level,match,alliance,team,station,actor,task,measuretype,phase,attempt,reason,capability,successes,attempts,cycle_times,last_match,num_matches
16,2019-03-30T12:46:00,test_event_2,2020,qual,013-q,blue,1318,1,robot,climb_Side,enum,finish,summary,na,Side,5,0,0,1,6
17,2019-03-30T12:46:00,test_event_2,2020,qual,013-q,blue,1318,1,robot,climb_Side,enum,finish,summary,na,Side,5,0,0,2,6
28,2019-03-30T12:28:00,test_event_2,2020,qual,011-q,red,1318,2,robot,climb_Center,enum,finish,summary,na,Center,5,0,0,3,6
29,2019-03-30T12:28:00,test_event_2,2020,qual,011-q,red,1318,2,robot,climb_Center,enum,finish,summary,na,Center,5,0,0,4,6
125,2019-03-30T11:52:00,test_event_2,2020,qual,007-q,blue,2046,3,robot,climb_Center,enum,finish,summary,na,Center,5,0,0,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2377,2019-03-30T14:14:00,test_event_2,2020,qual,016-q,red,948,3,robot,climb_Center,enum,finish,summary,na,Center,5,0,0,3,8
2381,2019-03-30T14:14:00,test_event_2,2020,qual,016-q,red,948,3,robot,climb_Center,enum,finish,summary,na,Center,5,0,0,4,8
2382,2019-03-30T11:34:00,test_event_2,2020,qual,005-q,blue,948,3,robot,climb_Center,enum,finish,summary,na,Center,5,0,0,8,8
2390,2019-03-30T11:34:00,test_event_2,2020,qual,005-q,blue,948,3,robot,climb_Center,enum,finish,summary,na,Center,5,0,0,7,8


In [9]:
data = ds.DataSource(event='test_event_2', season='2020')
oneteam = OneTeam(data)
otdf = oneteam.df_new_1t('948', ['shootUpper', 'shootLower', 'climbPosition'])
#otdf[otdf.task.isin(['climb_side', 'climb_center'])]
otdf

task,climb_center,climb_side,shootLower,shootUpper
match,,,,
005-q,10.0,0.0,14.0,32.0
011-q,0.0,10.0,20.0,0.0
016-q,10.0,0.0,8.0,16.0
